In [14]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np

In [27]:
x_train = np.load("x_train_out.npy")
y_train = np.load("y_train_out.npy")
x_test = np.load("x_test_out.npy")
y_test = np.load("y_test_out.npy")

In [28]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [29]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [30]:
#one hot encoding of target labels
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [31]:
baseMapNum = 32
weight_decay = 1e-4

In [32]:
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [33]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

In [34]:
batch_size = 64
data_augmentation = True

In [36]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=20,verbose=1,validation_data=(x_test,y_test))
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    

-------------Using Data augmentation------------
Epoch 1/20
781/781 [==============================] - 2013s - loss: 1.9023 - acc: 0.4254 - val_loss: 1.2061 - val_acc: 0.5941
Epoch 2/20
781/781 [==============================] - 1943s - loss: 1.2858 - acc: 0.5889 - val_loss: 0.9902 - val_acc: 0.6807
Epoch 3/20
781/781 [==============================] - 1911s - loss: 1.0342 - acc: 0.6666 - val_loss: 0.9119 - val_acc: 0.7058
Epoch 4/20
781/781 [==============================] - 1861s - loss: 0.9072 - acc: 0.7050 - val_loss: 0.8021 - val_acc: 0.7461
Epoch 5/20
781/781 [==============================] - 1896s - loss: 0.8422 - acc: 0.7314 - val_loss: 0.7770 - val_acc: 0.7564
Epoch 6/20
781/781 [==============================] - 1864s - loss: 0.7955 - acc: 0.7499 - val_loss: 0.7156 - val_acc: 0.7840
Epoch 7/20
781/781 [==============================] - 1847s - loss: 0.7587 - acc: 0.7654 - val_loss: 0.6748 - val_acc: 0.7956
Epoch 8/20
781/781 [==============================] - 1865s - loss: 0

In [37]:
model.save_weights("simple_cnn_adam_20.h5")

In [38]:
model.load_weights("simple_cnn_adam_20.h5")

In [39]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=20,verbose=1,validation_data=(x_test,y_test))
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    

-------------Using Data augmentation------------
Epoch 1/20
781/781 [==============================] - 1926s - loss: 0.5973 - acc: 0.8439 - val_loss: 0.5652 - val_acc: 0.8545
Epoch 2/20
781/781 [==============================] - 1947s - loss: 0.5953 - acc: 0.8455 - val_loss: 0.5526 - val_acc: 0.8639
Epoch 3/20
781/781 [==============================] - 1935s - loss: 0.5936 - acc: 0.8458 - val_loss: 0.5566 - val_acc: 0.8638
Epoch 4/20
781/781 [==============================] - 1953s - loss: 0.5848 - acc: 0.8496 - val_loss: 0.5774 - val_acc: 0.8564
Epoch 5/20
781/781 [==============================] - 1944s - loss: 0.5875 - acc: 0.8497 - val_loss: 0.5537 - val_acc: 0.8649
Epoch 6/20
781/781 [==============================] - 1964s - loss: 0.5821 - acc: 0.8515 - val_loss: 0.5619 - val_acc: 0.8600
Epoch 7/20
781/781 [==============================] - 1941s - loss: 0.5817 - acc: 0.8543 - val_loss: 0.5502 - val_acc: 0.8645
Epoch 8/20
781/781 [==============================] - 1948s - loss: 0

In [40]:
model.save_weights("cifar_simple_cnn_adam_40.h5")

In [12]:
model.load_weights("cifar10_90.62.h5")

10000/10000 [==============================] - 62s    

Test result: 90.620 loss: 0.379
